<a href="https://colab.research.google.com/github/Hoiy/tf_playground/blob/master/bert-chinese-tokenizer/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install pyarrow pandas sklearn
!mkdir -p fonts && cd fonts && curl -JLO http://fonts.cooltext.com/Downloader.aspx?ID=11120

    100% |████████████████████████████████| 11.6MB 118kB/s 
    100% |████████████████████████████████| 8.7MB 161kB/s 
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 13.8MB 102kB/s 
    100% |████████████████████████████████| 512kB 2.8MB/s 
  Using cached https://files.pythonhosted.org/packages/74/68/d87d9b36af36f44254a8d512cbfc48369103a3b9e474be9bdfe536abfc45/python_dateutil-2.7.5-py2.py3-none-any.whl
    100% |████████████████████████████████| 5.3MB 280kB/s 
    100% |████████████████████████████████| 26.5MB 48kB/s 
  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /home/Hoiy/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [0]:
!sudo apt-get install unzip
!wget http://sighan.cs.uchicago.edu/bakeoff2005/data/icwb2-data.zip
!unzip icwb2-data.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
--2019-01-02 13:36:54--  http://sighan.cs.uchicago.edu/bakeoff2005/data/icwb2-data.zip
Resolving sighan.cs.uchicago.edu (sighan.cs.uchicago.edu)... 128.135.164.125
Connecting to sighan.cs.uchicago.edu (sighan.cs.uchicago.edu)|128.135.164.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52640127 (50M) [application/zip]
Saving to: ‘icwb2-data.zip’

icwb2-data.zip      100%[===================>]  50.20M  21.0MB/s    in 2.4s    

2019-01-02 13:36:56 (21.0 MB/s) - ‘icwb2-data.zip’ saved [52640127/52640127]

Archive:  icwb2-data.zip
   creating: icwb2-data/
   creating: icwb2-data/doc/
  inflating: icwb2-data/doc/instructions.txt  
  inflating: icwb2-data/doc/result_instructions.txt  
   creating: icwb2-data/gold/
  inflating: icwb2-data/gold/as_testing_go

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

In [0]:
import os
import tokenization
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

from google.colab import auth
auth.authenticate_user()

In [0]:
BERT_PRETRAINED_DIR = 'gs://dev-test-bert-tpu/chinese_bert/'
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
BERT_MODEL = 'chinese_L-12_H-768_A-12'
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [0]:
# l = list(filter(None, train_examples[5].text_a.split(' ')))
# print(''.join(l))
# t = ['１' if i < len(token) - 1 else '在' for token in l for i in range(len(token))]
# print(''.join(t))

# [tokenizer.tokenize(item) for item in l]
# l[0] = '１１'
# tokenizer.tokenize(l[0])

# input: list of string, output: list of 0,1 to each char

# 1. Call tokenizer to joined list of string, note the new string size may increase or decrease
# 2. 

# raw_text_tokenized = '标价牌  上  的  数字  也  会  令  你  惊喜  ，  意大利  皮鞋  ，  最  便宜  的  只  需  １０  元  ，  好  的  也  不过  ５０  元  ；  一  件  泰国  产  鳄鱼衫  才  ３．５  元  。  可  仔细  一  盘算  ，  并非  那么  便宜  。  科威特  货币  叫  Ｋ·Ｄ  （  科威特  第纳尔  ）  ，  １  Ｋ·Ｄ  约  为  ３．３  美元  ，  面  小  值  大  。'
raw_text_tokenized = 'Ｂ７３７—３００  新世纪  ——  一  １１１１  ＫＫ·Ｄ  。  １２月  ３１日  。  １１００  。  ６—１２  。  Ｄ'

# Entry point for both training and testing set
token_list = list(filter(None,raw_text_tokenized.split(' ')))

def preprocess_tokens_list(token_list):
  raw_text_truths = [1. if i == len(token) - 1 else 0. \
                             for token in token_list \
                             for i in range(len(token))]
  raw_text_char_list = list(ch for ch in ''.join(token_list))

  assert len(raw_text_char_list) == len(raw_text_truths)
  return raw_text_char_list, raw_text_truths


# def search_list_end(char_list, tokens):
#   assert start_1 < len(list_1)
#   assert start_2 < len(list_2)
  
#   assert list_2[start_2] != '[UNK]'
  
#   if :
#     # ['UNK'] not the last element
#     assert start_2 + 1 < len(list_2)
#     if list_2[start_2 + 1] == '[UNK]':
#       return start_1 + 1, start_2 + 1
  
# B A C 1 0
# [UNK] [UNK] 10
  

def process_raw_text(raw_text_char_list, raw_text_truths):
  processed_text_char_list = tokenizer.tokenize(''.join(raw_text_char_list))
  processed_text_truths = [1.] * len(processed_text_char_list)
  
  j = 0
  i = 0
  while i < len(processed_text_char_list):
    
    def is_match(raw_text_char_list, i, processed_text_char):
      assert len(processed_text_char) >= 1
      if processed_text_char == '[UNK]':
        return True
      
      if processed_text_char.startswith('##'):
        processed_text_char = processed_text_char[2:]
      
      k = 0
      while k < len(processed_text_char):
        if raw_text_char_list[i+k] != processed_text_char[k]:
          return False
        k += 1
      return True
    
    
    if processed_text_char_list[i] == '[UNK]':
      if i+1 >= len(processed_text_char_list): # ['UNK'] is last element
        processed_text_truths[i] = raw_text_truths[-1]
        i+=1
        continue
        
      if processed_text_char_list[i+1] == '[UNK]': # more than 1 consecutive ['UNK']
        processed_text_truths[i] = raw_text_truths[j]
        i+=1
        j+=1
        continue
      
      # next char is not [UNK]
      if is_match(raw_text_char_list ,j, processed_text_char_list[i+1]):
        i+=1
        continue

      processed_text_truths[i] = raw_text_truths[j]
      j+=1
      continue
        
#     if is_match(raw_text_truths[j], processed_text_char_list[i]):
    
        
#       # only one [UNK]
#       assert raw_text_char_list[j], (raw_text_char_list, processed_text_char_list)
#       assert processed_text_char_list[i+1]
#       while raw_text_char_list[j] != processed_text_char_list[i+1]:
#         j += 1
#         assert j < len(raw_text_char_list), (raw_text_char_list, processed_text_char_list)

#       processed_text_truths[i] = raw_text_truths[j-1] # match to last j
#       i += 1
#       processed_text_truths[i] = raw_text_truths[j]
#       j += 1
#       i += 1
#       continue

    if len(processed_text_char_list[i]) > 1:
      k = 0
      
      # skip matching ##
      if processed_text_char_list[i][:2] == '##':
        k = 2
        
      while k < len(processed_text_char_list[i]):
        if processed_text_char_list[i][k] == raw_text_char_list[j]:
          j += 1
          k += 1
        else:
          assert False, (processed_text_char_list, raw_text_char_list, processed_text_char_list[i], raw_text_char_list[j])

      processed_text_truths[i] = raw_text_truths[j-1]
      i += 1
      continue

    if processed_text_char_list[i] == raw_text_char_list[j]:
      processed_text_truths[i] = raw_text_truths[j]
      i += 1
      j += 1
      continue

    assert False, (processed_text_char_list[i], raw_text_char_list[j])


  assert len(processed_text_char_list) == len(processed_text_truths)
  return processed_text_char_list, processed_text_truths


raw_text_char_list, raw_text_truths = preprocess_tokens_list(token_list)
(token_list, raw_text_char_list, raw_text_truths) + process_raw_text(raw_text_char_list, raw_text_truths)


  

# raw_text_char_list = list(ch for ch in ''.join(raw_text_list))
# processed_text_char_list = tokenizer.tokenize(''.join(raw_text_char_list))

# print()
# print(raw_text_char_list)
# print(processed_text_char_list)

# # origin_text = '１１１１  ＫＫ·Ｄ'


    
    

# process_raw_text_tokenized(raw_text_list)


# Assumption:
# All string in raw_text_char_list has length 1.
#
# Forward matching rules:
#
# For element in processed_text_list:
#   1. if it is '[UNK]', treated it as *, match until next symbol matched
#   2. otherwise, if it has length >= 1 and it matched the first char, match until all chars matched from raw_text_char_list
#   3. otherwise (all sort of unmatched, e.g. ##1 vs 1)


# j = 0
# for i in range(len(processed_text_list)):
#   if raw_text_list[j] == processed_text_list[i]:
#     processed_text_truths[i] = raw_text_truths[j]
#     j += 1
#     continue
    
# print(raw_text_tokenized)
# print(''.join(raw_text_char_list))
# print(''.join(raw_text_truths))
# print(''.join(processed_text_char_list))
# print(''.join(processed_text_truths))


# origin_text = list(filter(None, origin_text.split(' ')))
# tokens_a_truth = [1. if i == len(token) - 1 else 0. for token in text for i in range(len(tokenizer.tokenize(token)))]
# text_a = ''.join(origin_text)

# #   assert len(text_a) == len(tokens_a_truth)

# tokens_a = tokenizer.tokenize(text_a)
# tokens_a_truth = [1] * len(tokens_a) # default split all

# # for token in origin_text:
# #   if token in tokens_a

# print(tokens_a)
# print(tokens_a_truth)
# assert len(tokens_a) == len(tokens_a_truth), (len(tokens_a), len(tokens_a_truth), tokens_a, tokens_a_truth)


(['Ｂ７３７—３００',
  '新世纪',
  '——',
  '一',
  '１１１１',
  'ＫＫ·Ｄ',
  '。',
  '１２月',
  '３１日',
  '。',
  '１１００',
  '。',
  '６—１２',
  '。',
  'Ｄ'],
 ['Ｂ',
  '７',
  '３',
  '７',
  '—',
  '３',
  '０',
  '０',
  '新',
  '世',
  '纪',
  '—',
  '—',
  '一',
  '１',
  '１',
  '１',
  '１',
  'Ｋ',
  'Ｋ',
  '·',
  'Ｄ',
  '。',
  '１',
  '２',
  '月',
  '３',
  '１',
  '日',
  '。',
  '１',
  '１',
  '０',
  '０',
  '。',
  '６',
  '—',
  '１',
  '２',
  '。',
  'Ｄ'],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0],
 ['[UNK]',
  '[UNK]',
  '３０',
  '##０',
  '新',
  '世',
  '纪',
  '[UNK]',
  '[UNK]',
  '一',
  '[UNK]',
  '·',
  '[UNK]',
  '。',
  '１２',
  '月',
  '３',
  '##１',
  '日',
  '。',
  '１１',
  '##００',
  '。',
  '６',
  '[UNK]',
  '１２',
  '。',
  '[UNK]'],
 [0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0

In [0]:
data = pd.read_csv('./icwb2-data/training/pku_training.utf8', header=None)
display(data.head())

train, val = train_test_split(data, test_size=0.2, shuffle=False)
train.shape, val.shape

,0
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...
1,中共中央 总书记 、 国家 主席 江 泽民
2,（ 一九九七年 十二月 三十一日 ）
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：


((15243, 1), (3811, 1))

In [0]:
test = pd.read_csv('./icwb2-data/testing/pku_test.utf8', header=None)
test.head()

,0
0,共同创造美好的新世纪——二○○一年新年贺词
1,（二○○○年十二月三十一日）（附图片1张）
2,女士们，先生们，同志们，朋友们：
3,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...
4,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...


In [0]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

      
def get_examples(df):
  examples = []
  
  def append_example(row):   
    examples.append(InputExample(row.name, row[0]))

  df.apply(append_example, axis=1)
  return examples

train_examples = get_examples(train)
val_examples = get_examples(val)
test_examples = get_examples(test)
len(train_examples), len(val_examples), len(test_examples)

(15243, 3811, 1944)

In [0]:
# # For char count frequency
# keras_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, oov_token='<oov>', split='')
# keras_tokenizer.fit_on_texts(train.main_content)
# len(keras_tokenizer.index_word), keras_tokenizer.word_index['<oov>']

# choice = []
# p = []

# for k, v in keras_tokenizer.word_counts.items():
#   if k in tokenizer.vocab:
#     choice.append(k)
#     p.append(v)

# p = p / np.sum(p)

# choice[:10], p[:10]

In [0]:
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               truths,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.is_real_example = is_real_example
    self.truths = truths

    
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""
  
  token_list = list(filter(None, example.text_a.split(' ')))
  
  tokens_a, tokens_a_truth = process_raw_text(*preprocess_tokens_list(token_list))
    
  assert len(tokens_a) == len(tokens_a_truth)  
  
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  truths = []
  
  tokens.append("[CLS]")
  segment_ids.append(0)
  truths.append(1.)
  
  for token, truth in zip(tokens_a, tokens_a_truth):
    tokens.append(token)
    segment_ids.append(0)
    truths.append(truth)
    
  tokens.append("[SEP]")
  segment_ids.append(0)
  truths.append(1.)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)
    truths.append(1.)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)
    truths.append(0.)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length
  assert len(truths) == max_seq_length

  if ex_index < 3:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("truths: %s" % " ".join([str(x) for x in truths]))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      truths=truths
  )
  return feature


def convert_examples_to_features(examples, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 1000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features  

print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')

MAX_SEQ_LENGTH = 512 #@param {'type': 'number'}

train_features = convert_examples_to_features(train_examples, MAX_SEQ_LENGTH, tokenizer)
val_features = convert_examples_to_features(val_examples, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_features(test_examples, MAX_SEQ_LENGTH, tokenizer)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 15243
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] 迈 向 充 满 希 望 的 新 世 纪 [UNK] [UNK] 一 九 九 八 年 新 年 讲 话 （ 附 图 片 １ 张 ） [SEP]
INFO:tensorflow:input_ids: 101 6815 1403 1041 4007 2361 3307 4638 3173 686 5279 100 100 671 736 736 1061 2399 3173 2399 6382 6413 8020 7353 1745 4275 8029 2476 8021 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
!mkdir -p dataset

import pickle

with open('./dataset/train_features_%s.pkl'%MAX_SEQ_LENGTH, 'wb') as f:
  pickle.dump(train_features, f)

  
with open('./dataset/val_features_%s.pkl'%MAX_SEQ_LENGTH, 'wb') as f:
  pickle.dump(val_features, f)

  
with open('./dataset/test_features_%s.pkl'%MAX_SEQ_LENGTH, 'wb') as f:
  pickle.dump(test_features, f)

!gsutil rsync ./dataset gs://dev-test-bert-tpu/bert-chinese-tokenizer/dataset

Building synchronization state...
Starting synchronization...
Copying file://./dataset/test_features_512.pkl [Content-Type=application/octet-stream]...
Copying file://./dataset/train_features_512.pkl [Content-Type=application/octet-stream]...
Copying file://./dataset/val_features_512.pkl [Content-Type=application/octet-stream]...
/
Operation completed over 3 objects/156.9 MiB.                                    
